# Imports

In [ ]:

# import a library with an alias
import pandas as pd

# import a module from a package
from matplotlib import pyplot as plt


import seaborn as sns

In [ ]:


# File paths - Update these with your actual file locations
SHARE_PRICES_FILE = "data/simfin_data/us-shareprices-daily.csv"
COMPANY_INFO_FILE = "data/simfin_data/us-companies.csv"

# Load Share Prices Data
df_share_prices = pd.read_csv(SHARE_PRICES_FILE, delimiter=';', parse_dates=['Date'])
print("✅ Share Prices Data Loaded. Shape:", df_share_prices.shape)

# Load Company Info Data
df_company = pd.read_csv(COMPANY_INFO_FILE, delimiter=';')
print("✅ Company Info Data Loaded. Shape:", df_company.shape)


In [ ]:
df_share_prices.head()

In [ ]:
df_company.head()

# 2. Data transformation

## 2.1 Handiling columns name convention

In [ ]:
# Example transformations:
df_share_prices.columns = [col.strip().lower().replace(" ", "_") for col in df_share_prices.columns]  # Normalize column names
# Example transformations:
df_company.columns = [col.strip().lower().replace(" ", "_") for col in df_company.columns]  # Normalize column names

In [ ]:
df_share_prices['date'] = pd.to_datetime(df_share_prices['date'])  # Convert date columns


In [ ]:
df_share_prices.head()


In [ ]:
df_company.head()

## 2.2 Checking for null values

### 2.2.1 Share prices

In [ ]:
# Check for missing values
print(df_share_prices.isnull().sum())  # Count of nulls per column


plt.figure(figsize=(10,5))
sns.heatmap(df_share_prices.isnull(), cmap="viridis", cbar=False, yticklabels=False)
plt.title("Missing Values Heatmap")
plt.show()

In [ ]:
# Calculate percentage of missing values per column
null_percentage = (df_share_prices.isnull().sum() / len(df_share_prices)) * 100

# Display the result
print(null_percentage)

#### 2.2.1.1 Handle missing values

In [ ]:
# Cleaning Share Prices Data
def clean_share_prices(df):
    df = df.copy()  # Avoid chained assignment warnings
    df = df.drop(columns=['dividend'], errors='ignore')  # Drop the dividend column
    df = df.assign(
        shares_outstanding=df['shares_outstanding'].ffill()  # Forward fill missing shares outstanding
    )
    df = df.sort_values(by=['ticker', 'date'])
    return df


# Apply Cleaning Functions
df_share_prices_cleaned = clean_share_prices(df_share_prices)


# Check Results
print("✅ Share Prices Cleaned. Shape:", df_share_prices_cleaned.shape)

print("🔍 Missing Values in Share Prices:\n", df_share_prices_cleaned.isnull().sum())


### 2.2.1 Companies

In [ ]:
# Check for missing values
print(df_company.isnull().sum())  # Count of nulls per column


plt.figure(figsize=(10,5))
sns.heatmap(df_company.isnull(), cmap="viridis", cbar=False, yticklabels=False)
plt.title("Missing Values Heatmap")
plt.show()

In [ ]:
# Calculate percentage of missing values per column
null_percentage = (df_company.isnull().sum() / len(df_company)) * 100

# Display the result
print(null_percentage)

In [ ]:
# Cleaning Company Info Data
def clean_company_info(df):
    df = df.copy()  # Avoid chained assignment warnings
    
    # Drop rows where ticker is missing (since we need it for merging)
    df = df.dropna(subset=['ticker'])
    
    # Fill missing values
    df = df.assign(
        company_name=df['company_name'].fillna('Unknown'),
        industryid=df['industryid'].fillna(-1).astype('category'),  # Fill with -1 and convert to category
        end_of_financial_year_month=df['end_of_financial_year_(month)'].fillna(df['end_of_financial_year_(month)'].mode()[0]),  # Fill with most frequent value
        number_employees=df['number_employees'].fillna(df['number_employees'].median()),  # Fill missing with median
        business_summary=df['business_summary'].fillna('No summary available')
    )
    
    return df

# Apply Cleaning Function
df_company_cleaned = clean_company_info(df_company)

# Check Missing Values Again
print("✅ Company Info Cleaned. Shape:", df_company_cleaned.shape)
print("🔍 Missing Values in Company Info:\n", df_company_cleaned.isnull().sum())


In [ ]:
df_share_prices.info()

In [ ]:
df_company.info()

# 3. Merging

In [ ]:
# Merging Share Prices with Company Info
def merge_datasets(share_prices, company_info):
    merged_df = share_prices.merge(company_info, on='ticker', how='left')
    return merged_df

# Apply Merge
df_merged = merge_datasets(df_share_prices_cleaned, df_company_cleaned)

# Check Results
print("✅ Merged Data Shape:", df_merged.shape)
print("🔍 Missing Values After Merge:\n", df_merged.isnull().sum())


In [ ]:
df_merged

## 3.1 Final clean up

In [ ]:
print(df_merged.columns)


In [ ]:
# Drop redundant 'simfinid_y' column (keeping 'simfinid_x' as 'simfinid')
df_merged = df_merged.drop(columns=['simfinid_y']).rename(columns={'simfinid_x': 'simfinid'})



In [ ]:
# Save the processed dataset
output_path = "processed_data"
df_merged.to_csv(f"{output_path}.csv", index=False)
df_merged.to_parquet(f"{output_path}.parquet")

# Check final structure
print("✅ Final Processed Data Shape:", df_merged.shape)
print("🔍 Missing Values in Final Data:\n", df_merged.isnull().sum())
print("📂 Data saved successfully as CSV and Parquet!")


# 4. Preparing the Model Table Data

In [ ]:
def prepare_company_data(df, ticker):
    """
    Prepare model data for a specific company based on its ticker.
    """
    df_company = df[df['ticker'] == ticker].copy()  # Filter for selected company
    
    # Ensure data is sorted correctly
    df_company = df_company.sort_values(by='date')

    # Creating lag features (last 3 days closing prices)
    df_company['close_t-1'] = df_company['close'].shift(1)
    df_company['close_t-2'] = df_company['close'].shift(2)
    df_company['close_t-3'] = df_company['close'].shift(3)

    # Target Variable: Next day's price movement (1 if up, 0 if down)
    df_company['target'] = (df_company['close'].shift(-1) > df_company['close']).astype(int)

    # Drop NaN values caused by shifting
    df_company.dropna(inplace=True)

    return df_company[['date', 'close_t-3', 'close_t-2', 'close_t-1', 'target']]

# Specify a company ticker
selected_ticker = "AAPL"  # Change this to test different companies

# Apply the function
df_company_model = prepare_company_data(df_merged, selected_ticker)

# Check the processed data
print(f"✅ Data Prepared for {selected_ticker}. Shape:", df_company_model.shape)
print("🔍 Sample Data:\n", df_company_model.head())



# 5. Traing the model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# ✅ Ensure Data is Numeric (Fixing Categorical Columns)
def prepare_xgboost_data(df):
    """
    Ensure the dataset is numeric by dropping text columns and encoding categorical features.
    """
    # Drop non-numeric columns that don't contribute to the model
    cols_to_drop = ['ticker', 'company_name', 'isin', 'business_summary', 'market', 'main_currency']
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns], errors='ignore')

    # Convert categorical columns to numeric (if needed)
    if 'industryid' in df.columns:
        df['industryid'] = df['industryid'].astype('category').cat.codes  # Convert to numeric codes

    return df

# Apply the fix before training
df_company_model_fixed = prepare_xgboost_data(df_company_model)

print("✅ Data Preparation Complete. Shape:", df_company_model_fixed.shape)


In [ ]:


# Set SimFin API Key (Replace with your actual key)
sf.set_api_key('344dd533-861f-4bef-9f52-be02f0276014')

# Set SimFin Data Directory (for caching data)
sf.set_data_dir('simfin_data')

def get_latest_stock_data(ticker):
    """
    Fetch the latest available stock data (Close Price, Volume, etc.) from SimFin.
    """
    try:
        # Fetch daily stock price data
        df = sf.load_shareprices(variant='daily', market='us')

        # Ensure 'Ticker' exists in the index
        if ticker not in df.index.get_level_values(0):
            print(f"❌ No data found for ticker: {ticker}")
            return None

        # Select only the specified ticker using index filtering
        df_ticker = df.loc[ticker].copy()

        # Get the latest row
        latest_row = df_ticker.iloc[-1]  

        # Extract required features
        latest_data = {
            'close_t-1': df_ticker['Close'].shift(1).iloc[-1],
            'close_t-2': df_ticker['Close'].shift(2).iloc[-1],
            'close_t-3': df_ticker['Close'].shift(3).iloc[-1],
            'close_t-4': df_ticker['Close'].shift(4).iloc[-1],
            'close_t-5': df_ticker['Close'].shift(5).iloc[-1],
            'vol_ma_5': df_ticker['Volume'].rolling(window=5).mean().iloc[-1],
            'vol_ma_10': df_ticker['Volume'].rolling(window=10).mean().iloc[-1],
            'day_of_week': latest_row.name.dayofweek,  # Monday=0, Sunday=6
            'month': latest_row.name.month
        }

        # Convert to DataFrame (for prediction)
        latest_df = pd.DataFrame([latest_data])

        print("✅ Latest Stock Data Fetched from SimFin!")
        return latest_df

    except Exception as e:
        print(f"❌ Error Fetching Data from SimFin: {e}")
        return None

# Test with a known stock symbol
get_latest_stock_data("A")




In [ ]:
def train_xgboost_final(df_company_model):
    """
    Train an optimized XGBoost model using RandomizedSearchCV for better accuracy.
    """
    # Ensure 'target' column exists
    if 'target' not in df_company_model.columns:
        print("❌ Error: 'target' column is missing. Recalculating it now.")
        df_company_model['target'] = (df_company_model['close'].shift(-1) > df_company_model['close']).astype(int)

    # Splitting Features (X) and Target (y)
    cols_to_drop = ['date', 'close', 'target']
    cols_to_drop = [col for col in cols_to_drop if col in df_company_model.columns]  # Drop only if they exist

    X = df_company_model.drop(columns=cols_to_drop)
    y = df_company_model['target']

    # Train-Test Split (80% Train, 20% Test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Define XGBoost Model
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

    # Final Optimized Hyperparameters
    param_dist = {
        'n_estimators': [200, 300, 400],
        'learning_rate': [0.05, 0.1, 0.15],
        'max_depth': [3, 5],
        'min_child_weight': [1, 3],
        'gamma': [0, 0.1, 0.2],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.6, 0.8],
        'scale_pos_weight': [1.2, 1.5]  # Adjusting for class imbalance
    }

    # Perform Randomized Search
    rand_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=15,
        scoring='accuracy', cv=3, n_jobs=-1, random_state=42
    )

    rand_search.fit(X_train, y_train)

    # Best Model
    best_model = rand_search.best_estimator_

    # Make Predictions
    y_pred = best_model.predict(X_test)

    # Evaluate Performance
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print("✅ Final XGBoost Model Training Complete!")
    print(f"📊 Best Hyperparameters: {rand_search.best_params_}")
    print(f"📊 Final Optimized Accuracy: {accuracy:.4f}")
    print("\n🔍 Classification Report:\n", report)

    return best_model

# Train the final optimized model
xgb_model_final = train_xgboost_final(df_company_model_fixed)


In [ ]:
import joblib

# Save the model as a binary file
joblib.dump(xgb_model_final, "xgb_model_final.pkl")


print("✅ Model saved successfully!")


In [ ]:
# Print features used during model training
print("✅ Features Used in Training:", xgb_model_final.feature_names_in_)

# Fetch latest stock data
latest_data = get_latest_stock_data("AAPL")

# Print features from API data
print("✅ Features from SimFin API:", latest_data.columns if latest_data is not None else "No data fetched")


In [ ]:
def predict_next_day_xgboost_api(model, ticker):
    """
    Fetch latest stock data from SimFin and predict whether the stock will go up or down.
    """
    # Fetch latest stock data from SimFin API
    latest_data = get_latest_stock_data(ticker)

    if latest_data is None:
        print("❌ Could not retrieve latest stock data. Aborting prediction.")
        return

    # Ensure feature alignment: Keep only the features used in training
    model_features = model.feature_names_in_
    latest_data = latest_data[model_features]  # Select only relevant columns

    # Make Prediction
    prediction = model.predict(latest_data)
    prediction_label = "📈 Up" if prediction[0] == 1 else "📉 Down"

    print(f"🔮 XGBoost Prediction for {ticker}: {prediction_label}")

# Example Usage: Predict for AAPL
predict_next_day_xgboost_api(xgb_model_final, "AAPL")


## 5.1 Spliting testing and training

In [ ]:
import requests

class SimFinAPI:
    BASE_URL = "https://simfin.com/api/v2/companies/prices/list"
    API_KEY = "344dd533-861f-4bef-9f52-be02f0276014"  # Replace with your actual SimFin API key

    def __init__(self):
        """Initialize API wrapper with headers."""
        self.headers = {
            "Authorization": f"api-key {self.API_KEY}"
        }

    def get_latest_stock_data(self, ticker):
        """
        Fetch the latest available stock data (Close Price, Volume, etc.) from SimFin.
        """
        try:
            # Define request parameters
            params = {
                "ticker": ticker,
                "timeframe": "daily",
                "start": "2024-01-01",  # Adjust as needed
                "end": "2024-12-31"
            }

            # Send API request
            response = requests.get(self.BASE_URL, headers=self.headers, params=params)

            # Print Full Response for Debugging
            print("🔍 Full API Response:", response.status_code, response.text)

            # Handle errors
            if response.status_code != 200:
                print(f"❌ Error Fetching Data: {response.json().get('error', 'Unknown error')}")
                return None

            # Parse JSON response
            data = response.json()

            print("✅ API Request Successful!")
            return data  # Return raw data for now

        except Exception as e:
            print(f"❌ Exception Fetching Data from SimFin: {e}")
            return None

# Initialize API Wrapper
simfin_api = SimFinAPI()

# Test with a known ticker (AAPL)
api_response = simfin_api.get_latest_stock_data("AAPL")
print("🔍 Raw API Response:", api_response)


In [ ]:
import requests

url = "https://backend.simfin.com/api/v3/companies/common-shares-outstanding?ticker=AAPL&start=2025-01-01"

headers = {
    "accept": "application/json",
    "Authorization": "api-key 344dd533-861f-4bef-9f52-be02f0276014"
}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
import requests
import pandas as pd

# Set your API key
API_KEY = "344dd533-861f-4bef-9f52-be02f0276014"

# Define the endpoint (use company ID, not ticker)
company_id = 59265  # Example: Apple Inc.'s ID on SimFin
url = f"https://simfin.com/api/v2/companies/id/{company_id}/shares/prices"

# Define the parameters
params = {
    "api-key": API_KEY,
    "start": "2023-01-01",  # Replace with your desired start date
    "end": "2024-03-11",  # Replace with your desired end date
    "timeframe": "d"  # 'd' for daily, 'w' for weekly, 'm' for monthly
}

# Make the GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Convert response to JSON
    if 'data' in data:  # Ensure there is data in response
        df = pd.DataFrame(data['data'], columns=data['columns'])  # Convert to DataFrame
        print(df.head())  # Print the first few rows
    else:
        print("No data found in response.")
else:
    print("Error:", response.status_code, response.text)


## 5.2 Standarize data

## 5.3 Training using logistic regresion model


## 5.4 Predict

## 5.5 Evaluation of model

In [7]:
import api_functions as api
simfin = api.SimFinAPI('AAPL')

# Fetch company info and stock data (defaults to last two weeks)
simfin.fetch_company_info()
simfin.fetch_stock_data()  # Uses last two weeks by default

df_company = simfin.get_company_dataframe()
df_stock = simfin.get_stock_dataframe()

✅ Company Info Retrieved for AAPL
✅ Company Data Processed for AAPL
2025-02-12
✅ Stock Price Data Retrieved for AAPL (from 2025-02-12)
✅ Stock Data Processed for AAPL


In [8]:
df_company

,companyDescription,endFy,id,industryName,isin,market,name,numEmployees,sectorCode,sectorName,ticker
0,Apple Inc is an American multinational technol...,9,111052,Technology,US0378331005,US,APPLE INC,147000,101001,Computer Hardware,AAPL


In [9]:
df_stock

,Date,Dividend Paid,Common Shares Outstanding,Last Closing Price,Adjusted Closing Price,Highest Price,Lowest Price,Opening Price,Trading Volume
0,2025-02-12,None,15022073000,236.87,236.87,236.96,230.68,231.20,44996462
1,2025-02-13,None,15022073000,241.53,241.53,242.34,235.57,236.91,52138207
2,2025-02-14,None,15022073000,244.60,244.60,245.55,240.99,241.25,40185610
3,2025-02-18,None,15022073000,244.47,244.47,245.18,241.84,244.15,48111486
4,2025-02-19,None,15022073000,244.87,244.87,246.01,243.16,244.66,32140384
5,2025-02-20,None,15022073000,245.83,245.83,246.78,244.29,244.94,31037471
6,2025-02-21,None,15022073000,245.55,245.55,248.69,245.22,245.95,53011988
7,2025-02-24,None,15022073000,247.10,247.10,248.86,244.42,244.93,51031845
8,2025-02-25,None,15022073000,247.04,247.04,250.00,244.91,248.00,47275651
9,2025-02-26,None,15022073000,240.36,240.36,244.98,239.13,244.33,44097533
